Este agarra los datos del csv con la matriz de grayscale de cada espectrograma, y se los pasa al modelo de CNN de PyTorch (definido en aux file models.py), los guarda para poder llamarlo sin re-entrenar desde otros archivos

In [ ]:
import os, sys
import pandas as pd
import numpy as np
import cupy as cp
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import util, models

In [5]:
# UNCOMMENT THE ONE BEING USED

# NOT NOISE-REDUCED
df = pd.read_csv(os.path.join('..', 'database', 'meta', 'final', 'train_data.csv'))

# NOISE-REDUCED
# df = pd.read_csv(os.path.join('..', 'database', 'meta', 'final', 'train_data2.csv'))

In [ ]:
# Extract labels and pixel values
labels = df['label'].values.astype(np.int64)
features = df.drop(columns=['label']).values.astype(np.float32)
features /= 255.0
# features = features.reshape(-1, 1, 313, 224) # TODO Try Uncommenting Me and comment the line below instead
features = features.reshape(-1, 1, 224, 313)

In [ ]:
print(features[:3])

In [ ]:
X = torch.tensor(features, dtype=torch.float32)
y = torch.tensor(labels, dtype=torch.long)

dataset = TensorDataset(X, y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(torch.cuda.get_device_name(device))
else:
    print("CUDA not available")

Using device: cuda
NVIDIA GeForce RTX 5080


In [ ]:
model = BirdCNN(num_classes=28).to(device) # import from models.py
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        preds = model(batch_X)
        loss = criterion(preds, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

### Pedirle a Chat que haga un Snippet de Train & Val

### Que muestre la Cross-Entropy Loss Curve, y la Accuracy

#### Hacer un training y mostrar accuracy, cross-entropy curve, decirle a chat. El tensor X, ya tiene toda la info de todas las muestras tomadas con panda dataframe. Mostrarle las celdas de arriba a Chat y va a entender lo que estamos haciendo. Decirle que nos edite esta ultima celda de arriba para hacer un training & val que muestre las metricas. 

#### Definir el modelo en utils/models.py, y reemplazar BirdCNN por models.BirdCNN (BirdCNN siendo el modelo definido en models.py)

para despues (para test)

#### Use SK Learn Metrics

#### Creo que la mejor forma para el test set es recorrer uno por uno los audios de test/, hacerles la pipeline, a ir acumulando las metrics.